In [ ]:
# Header Files
import pandas as pd
import csv
import numpy as np
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from csv import DictReader
from efficient_apriori import apriori as eff_app
import ast
from mlxtend.preprocessing import TransactionEncoder
from nltk.stem import WordNetLemmatizer
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from itertools import chain
import nltk
import random
import pickle

### Step 1.  Data Preprocessing ###

In [ ]:
#Function to stem and tokenize the recipes
def stem_tokens(tokens):
    lemmatizer = WordNetLemmatizer() 
    stemmed_items = []
    for items in tokens:
        stemmed_items.append(lemmatizer.lemmatize(items))
    return stemmed_items

def tokenize(text):    
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens)
    return stems

In [ ]:
#Load the kaggle dataset
# Create URL to JSON file (alternatively this can be a filepath)
url = 'data/train.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient='columns')

ingredients_1 = df['ingredients']
# View the first ten rows
train_list = ingredients_1.values.tolist()
print(train_list[0:10])

kaggle_cuisine_set = []
for ingr in ingredients_1:
    ingrStem = [' '.join(stem_tokens(tokenize(w))) for w in ingr]
    kaggle_cuisine_set.append(ingrStem)

In [ ]:
#Load the Schmidt Data Set
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

with np.load('data/simplified-recipes-1M.npz') as data:
    recipes = data['recipes']
    ingredients = data['ingredients']

schmitd_set = [] #name this to recipe list
#counter = 0
for recipe in recipes:
    if(recipe.size) == 0:
        continue
    schmitd_set.append(ingredients[recipe])

In [ ]:
#Remove 'bad' ingredients such as 'sliced, prepared, etc'
badingr = []

file1 = open('data/baddata.txt', 'r') 
Lines = file1.readlines() 
for line in Lines: 
    badingr.append(line.strip())

final_set_badwords= schmitd_set + kaggle_cuisine_set
final_clean_data = []
for recipe in final_set_badwords:
    temprecipe = []
    for ingr in recipe:
        if ingr in badingr:
            continue
        temprecipe.append(ingr)
    final_clean_data.append(temprecipe)

print(len(final_clean_data))

In [ ]:
#Dividing data into training and test sets

final_training_data = [] #training set to run with apriori
final_test_data = []     #represents ground truth

for row in final_clean_data:   #each row is a recipe i.e list of ingredients
    if len(row) == 0:
        continue    
    test_data = row.pop(random.randrange(len(row)))   #pops out random ingredients from the row
    final_test_data.append(test_data)
    final_training_data.append(row)
    
print(len(final_test_data))
print(len(final_training_data))

### Step 2.  Association Rule Mining ###

In [ ]:
# Efficient Appriori
itemsets, rules = eff_app(final_training_data, min_support=0.08,  min_confidence=0.08)
print("Rules =>", rules)
type(rules)

In [ ]:
#pickle an object and save it to a file
pickle.dump(rules, open("pickledRules", "wb"))

#reconstruct the pickled object
reconstructedRules = pickle.load(open("pickledRules", "rb"))
print(type(reconstructedRules[0]))

### Step 3.  Calculating Precision ###

In [ ]:
#accessing the rules
testIngrList = ['salt']

fileObj =  open("pickledRules", "rb")
newRules = pickle.load(fileObj)

topKFinding = []

#for testIngr in testIngrList:
for r in newRules:
    if set(testIngrList) == set(list(r.lhs)):
        #Add Tuple of RHS and confidence to topKFinding
        topKFinding.append((list(r.rhs), r.confidence))

#Sort top k ingredients mapped from ARM in decreasing order of confidence
topKFinding.sort(key = lambda x: -x[1]) 

k = 5

topKFinding = topKFinding[0:k]
print(topKFinding)

In [ ]:
index_found = 0
map_ingr = {}

# Counts no of times an ingr is found in ground truth
for top_ingr in topKFinding:
    map_ingr[top_ingr[0][0]] = 0

for recipe in final_training_data:    
    if set(testIngrList).issubset(set(recipe)):
        #Checks if ingredient found in ground truth is actually what we predicted
        if final_test_data[index_found] in map_ingr:
            #print(final_test_data[index_found])
            #if yes then increase count            
            map_ingr[final_test_data[index_found]] += 1
    index_found += 1

print(map_ingr)

In [ ]:
#Calucating ARHR
k = 1
hits = 0
summation = 0

for top_ingr in topKFinding:
    # if there was a hit for rank k-th item
    if map_ingr[top_ingr[0][0]] > 0:     
        summation += 1.0 / k 
        hits += 1
    k += 1

arhr = summation/hits
print(arhr)

In [ ]:
#Calucating Average Precision
k = 1
hits = 0
summation = 0

for top_ingr in topKFinding:
    # if there was a hit for rank k-th item
    if map_ingr[top_ingr[0][0]] > 0:     
        #print(hits + 1, k)
        summation += (hits + 1) / k 
        hits += 1
    k += 1

ap = summation/hits
print(ap)